In [1]:
from pathlib import Path
from datetime import datetime, timedelta

import pandas as pd
import os
import json
import polars as pl
import matplotlib.pyplot as plt


ROOT_DIR = Path(os.getcwd()).parent

In [12]:
df_pumps: pd.DataFrame = pd.read_csv(
    os.path.join(ROOT_DIR, "data/pumps/pump_dumps_bitcoin_era.csv")
)

df_pumps[
    (df_pumps["exchange"] == "binance") &
    (df_pumps["group"] == "BPG")
]

,symbol,group,date,hour,exchange
253,ICN,BPG,2018-08-08,16:00,binance
254,OAX,BPG,2018-08-17,16:00,binance
255,RDN,BPG,2018-08-21,16:00,binance
256,DLT,BPG,2018-08-23,16:00,binance
257,RDN,BPG,2018-09-04,15:30,binance
258,AST,BPG,2018-09-07,15:30,binance
259,ICN,BPG,2018-09-11,15:30,binance
260,OAX,BPG,2018-09-25,14:00,binance
261,HC,BPG,2018-09-27,14:00,binance
262,BRD,BPG,2018-10-08,14:00,binance
